In [33]:
import os
import pandas as pd


In [34]:
import os
os.chdir('c:\\Users\\Vigneswaran\\OneDrive - Datanetiix Solutions Inc\\Desktop\\Diesease_Prognosis')

In [35]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))


In [36]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path  ## typically we have (accuracy, precision, recall, F1 score, etc.)
    target_column: str

In [37]:
from diseaseprognosis.constants import *
from diseaseprognosis.utils.common import read_yaml, create_directories,save_json

In [38]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [39]:

import os
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from urllib.parse import urlparse
import numpy as np
import joblib

In [40]:

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        conf_matrix = confusion_matrix(actual, pred)
        class_report = classification_report(actual, pred, output_dict=True)
        return accuracy, conf_matrix, class_report

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]  # No need for double brackets

        predicted_classes = model.predict(test_x)

        accuracy, conf_matrix, class_report = self.eval_metrics(test_y, predicted_classes)

        # Saving metrics as local
        scores = {
            "accuracy": accuracy,
            "confusion_matrix": conf_matrix.tolist(),  # Convert to list for JSON serialization
            "classification_report": class_report
        }
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [41]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-10-25 16:44:11,321: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-25 16:44:11,325: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-25 16:44:11,343: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-25 16:44:11,346: INFO: common: created directory at: artifacts]
[2024-10-25 16:44:11,348: INFO: common: created directory at: artifacts/model_evaluation]
[2024-10-25 16:44:11,489: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\Vigneswaran\OneDrive\ML_Project\venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Vigneswaran\OneDrive\ML_Project\venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
